# <span style="color:darkblue"> Lecture 18 - Connecting to SQL via Python </span>

<font size = "5">

In the past lectures:

- We worked directly in SQL
- Imported data
- Carried out basic dataset operations

In  this lecture we will run SQL from Python

- Combine the best of both!
- Learn about escape characters!

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Import libraries

In [12]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
import psycopg2
import sqlalchemy as sa
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Import Data

In [2]:
# Import the two car racing datasets

results = pd.read_csv("data_raw/results.csv")
races = pd.read_csv("data_raw/races.csv")
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Connect to SQL server

In [3]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details

url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = '12345')

con = sa.create_engine(url_server).connect()


<font size = "5">

Upload data to SQL

In [5]:
# Import the "results" table, with the name "results_sql"
# Con is an argument to specify the server connection
# "if_exists" is an option to replace the table if it already exists
# You can choose any name instead of "results_sql"

results.to_sql('results_sql',
               con = con,
               if_exists='replace')

# Import "races"
races.to_sql('races_sql',con, if_exists='replace')

102

<font size = "5">

Try it yourself!

- Upload the "circuits table" into SQL <br>
using ".to_sql()"

In [4]:
# Write your own code

circuits.to_sql('circuits_sql',con = con,if_exists = 'replace')

77

# <span style="color:darkblue"> II. "Escape" Characters </span>

<font size = "5">

- Backslash (\\) is known as an escape character
- Used for special operations on strings
- Great for working with text!

<font size = "5">

Multi-line strings (" \ ")

In [6]:
# Use a backslash to define strings over multiple lines
# Do not include a space after "\", otherwise it won't work.
example_string = "This is a string \
                  defined over multiple lines"
example_string

'This is a string                   defined over multiple lines'

<font size = "5">

"Double quotes" inside string

In [7]:
# Use a backslash + quotation 

example_double = "This will \"supposedly\" put double quotes inside a string"
print(example_double)

This will "supposedly" put double quotes inside a string


<font size = "5" >
'Single quotes' inside string

In [10]:
# There is no need for a backslash given single quotes 

example_single = "This will 'supposedly' put single quotes inside a string"
print(example_single)

This will 'supposedly' put single quotes inside a string


<font size = "5">

Try it yourself!

- Print a string ``` SELECT "driverId" FROM results; ``` <br> 
using backslash

In [8]:
# Write your own code

print("SELECT \"driverId\" FROM results;")



SELECT "driverId" FROM results;


# <span style="color:darkblue"> III. Data operations </span>

<font size = "5">

- In SQL, operations are often called "queries"


<font size = "5">

Import Data from SQL

In [10]:
# Use "pd.read_sql()"
# - The first argument is string with instructions wrapped in text()
#- The second argument is the server connection

# Extract all data from a column
example1 = pd.read_sql(text("SELECT * FROM results_sql;"),con)

# Extract a subset of columns
example2 = pd.read_sql(text("SELECT points \
                             FROM results_sql;"),con)

# Subset based on a string condition
example3 = pd.read_sql(text("SELECT * \
                             FROM races_sql \
                             WHERE name = 'Abu Dhabi Grand Prix';"),con)

# Note: (i) Remember to include "\"  to be able to define
#           strings over multiple lines
#       (ii) We can include single quotations in the WHERE command
#            without any additional escape characters

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: SELECT * FROM results_sql;]
(Background on this error at: https://sqlalche.me/e/20/2j85)

<font size = "5">

Upper case columns

- In SQL syntax we use double quotes e.g. ``` "driverId" ```
- ```.read_sql()``` requires a string inside a string
- To do so, use escape characters, e.g.   ``` \"driverId\" ```

In [ ]:
# Select a column
example4 = pd.read_sql(text("SELECT \"driverId\" \
                             FROM results_sql;"),con)

# Compute an aggregate statistic
example5 = pd.read_sql(text("SELECT AVG(points) as mean_points \
                             FROM results_sql \
                             GROUP BY \"driverId\" ;"),con)


<font size = "5">

Merge two datasets

In [17]:
# Merge 
# Reference the column \"driverId\" with escape characters
example6 = pd.read_sql(text("SELECT * \
                             FROM results_sql \
                             LEFT JOIN races_sql \
                             ON results_sql.\"raceId\" = races_sql.\"raceId\" ;"),con)

# Merge a subset of columns
# Use "results_sql.*" to select all columns from the primary dataset
# Use "races_sql.date" to only select the "date" column from the secondary dataset 

example7 = pd.read_sql(text("SELECT results_sql.*, races_sql.date \
                             FROM results_sql \
                             LEFT JOIN races_sql \
                             ON results_sql.\"raceId\" = races_sql.\"raceId\" ;"),con)


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- FROM results_sql compute the sum of points by <br>
"raceId" <br>

In [18]:
# Write your own code

pd.read_sql(text("SELECT SUM(points) AS sum_points \
                 FROM results_sql \
                 GROUP BY \"raceId\";"),con)



,sum_points
0,25.0
1,26.0
2,39.0
3,101.0
4,101.0
...,...
1074,39.0
1075,39.0
1076,26.0
1077,24.0


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- Merge "races_sql" and the circuits table on "circuitId" <br>
tha you imported above using ```LEFT JOIN```

In [20]:
# Write your own

example8 = pd.read_sql(text("SELECT * \
                            FROM races_sql \
                            LEFT JOIN circuits_sql \
                            ON races_sql.\"circuitId\" = circuits_sql.\"circuitId\" ;"),con)



# <span style="color:darkblue"> IV. Additional Material </span>

<font size = "5">

More about SQL syntax

https://www.w3schools.com/sql/